In [1]:
from copy import deepcopy

import pandas as pd

from BitBoard import osero
from osero_learn import learn

PLAY_WAY = deepcopy(osero.PLAY_WAY)
del PLAY_WAY["human"]
PLAY_WAY = PLAY_WAY.values()

In [2]:
eva = [[
     1.0, -0.6,  0.6,  0.4,  0.4,  0.6, -0.6,  1.0,
    -0.6, -0.8,  0.0,  0.0,  0.0,  0.0, -0.8, -0.6,
     0.6,  0.0,  0.8,  0.6,  0.6,  0.8,  0.0,  0.6,
     0.4,  0.0,  0.6,  0.0,  0.0,  0.6,  0.0,  0.4,
     0.4,  0.0,  0.6,  0.0,  0.0,  0.6,  0.0,  0.4,
     0.6,  0.0,  0.8,  0.6,  0.6,  0.8,  0.0,  0.6,
    -0.6, -0.8,  0.0,  0.0,  0.0,  0.0, -0.8, -0.6,
     1.0, -0.6,  0.6,  0.4,  0.4,  0.6, -0.6,  1.0
] for i in range(2)]

In [3]:
df = pd.DataFrame({})

check_point = [i for i in range(5, 64, 5)]

for i in range(10):
    print("\r[" + "#" * (i+1) + " " * (10-i+1) + "]", end="")
    for black in PLAY_WAY:
        for white in PLAY_WAY:
            run = learn(\
                black,
                white,
                check_point=check_point,
                seed_num=i,
                eva=eva
            )
            data = run.play()
            df = df.append(data, ignore_index=True)

print("\r[" + "#" * 10 + "]")

[##########]


In [4]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Ridge

x_data = df.drop(["turn", "last_score"], axis=1)
y_data = df[["turn_num", "last_score"]]

turn_vari = df["turn_num"].unique()

In [12]:
def two_plot(x, y, xlabel, ylabel, title, save_dir):
    fig = plt.figure(figsize=(10, 10))
    plt.plot(x, y[0], label="train MAE")
    plt.plot(x, y[1], label="test MAE")
    plt.minorticks_on()
    plt.grid(which="major")
    plt.grid(which="minor", linestyle="--")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.savefig(save_dir + "/" + title)
    plt.clf()
    plt.close()

In [40]:
model = []
alpha_arr = [i for i in range(3, 13)]

train_MAE = []
test_MAE = []

for turn_num in turn_vari:
    model.append([])

    for alpha in alpha_arr:
        x_train, x_test, y_train, y_test = train_test_split(\
            x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            random_state=0
        )

        model[-1].append(Ridge(\
            alpha=alpha,
            solver="lbfgs",
            positive=True,
            random_state=0
        ))
        model[-1][-1].fit(x_train, y_train)

        if alpha == alpha_arr[0]:
            train_predict = model[-1][-1].predict(x_train)
            test_predict = model[-1][-1].predict(x_test)
        else:
            train_predict += model[-1][-1].predict(x_train)
            test_predict += model[-1][-1].predict(x_test)

    train_predict = train_predict / len(alpha_arr)
    test_predict = test_predict / len(alpha_arr)

    train_MAE.append(mean_absolute_error(train_predict, y_train))
    test_MAE.append(mean_absolute_error(test_predict, y_test))

two_plot(turn_vari, [train_MAE, test_MAE], "turn", "MAE", "MAE of mean", "fig")

In [36]:
model = []
alpha_arr = [i for i in range(3, 13)]

train_MAE = []
test_MAE = []

for turn_num in turn_vari:
    train_predict = []
    test_predict = []

    model.append([])

    for alpha in alpha_arr:
        x_train, x_test, y_train, y_test = train_test_split(\
            x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            random_state=0
        )

        model[-1].append(Ridge(\
            alpha=alpha,
            solver="lbfgs",
            positive=True,
            random_state=0
        ))
        model[-1][-1].fit(x_train, y_train)

        train_predict.append(np.array(model[-1][-1].predict(x_train)))
        test_predict.append(np.array(model[-1][-1].predict(x_test)))
    
    train_predict = np.array(train_predict, dtype=np.int32).T
    test_predict = np.array(test_predict, dtype=np.int32).T

    for i in range(len(alpha_arr)):
        for j in range(len(train_predict[0][i])):
            train_predict[0][i][j] = np.round(train_predict[0][i][j])
        for j in range(len(test_predict[0][i])):
            test_predict[0][i][j] = np.round(test_predict[0][i][j])

    train_predict_mode = []
    test_predict_mode = []

    for i in range(len(x_train)):
        discard, unique = np.unique(train_predict[0][i], return_counts=True)
        train_predict_mode.append(np.argmax(unique))
    for i in range(len(x_test)):
        discard, unique = np.unique(test_predict[0][i], return_counts=True)
        test_predict_mode.append(np.argmax(unique))

    train_MAE.append(mean_absolute_error(train_predict_mode, y_train))
    test_MAE.append(mean_absolute_error(test_predict_mode, y_test))

two_plot(turn_vari, [train_MAE, test_MAE], "turn", "MAE", "MAE of mode", "fig")

In [38]:
model = []
alpha_arr = [i for i in range(3, 13)]
center = len(alpha_arr) // 2

train_MAE = []
test_MAE = []

for turn_num in turn_vari:
    train_predict = []
    test_predict = []
    model.append([])

    for alpha in alpha_arr:
        x_train, x_test, y_train, y_test = train_test_split(\
            x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            random_state=0
        )

        model[-1].append(Ridge(\
            alpha=alpha,
            solver="lbfgs",
            positive=True,
            random_state=0
        ))
        model[-1][-1].fit(x_train, y_train)
        train_predict.append(model[-1][-1].predict(x_train))
        test_predict.append(model[-1][-1].predict(x_test))
    
    train_predict = np.array(train_predict).T
    test_predict = np.array(test_predict).T

    train_predict_median = []
    test_predict_median = []

    for i in range(len(x_train)):
        sorted = np.sort(train_predict[0][i])
        train_predict_median.append(sorted[center])
    for i in range(len(x_test)):
        sorted = np.sort(test_predict[0][i])
        test_predict_median.append(sorted[center])

    train_MAE.append(mean_absolute_error(train_predict_median, y_train))
    test_MAE.append(mean_absolute_error(test_predict_median, y_test))

two_plot(turn_vari, [train_MAE, test_MAE], "turn", "MAE", "MAE of median", "fig")

In [ ]:
model = []
alpha_arr = [i for i in range(3, 13)]

train_MAE = []
test_MAE = []

for turn_num in turn_vari:
    model.append([])

    for alpha in alpha_arr:
        x_train, x_test, y_train, y_test = train_test_split(\
            x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
            random_state=0
        )

        model[-1].append(Ridge(\
            alpha=alpha,
            solver="lbfgs",
            positive=True,
            random_state=0
        ))
        model[-1][-1].fit(x_train, y_train)

        if alpha == alpha_arr[0]:
            train_predict = 1 / model[-1][-1].predict(x_train)
            test_predict = 1 / model[-1][-1].predict(x_test)
        else:
            train_predict += 1 / model[-1][-1].predict(x_train)
            test_predict += 1 / model[-1][-1].predict(x_test)

    train_predict = len(alpha_arr) / train_predict
    test_predict = len(alpha_arr) / test_predict

    train_MAE.append(mean_absolute_error(train_predict, y_train))
    test_MAE.append(mean_absolute_error(test_predict, y_test))

two_plot(turn_vari, [train_MAE, test_MAE], "turn", "MAE", "MAE of harmonic mean", "fig")

In [67]:
# weighted mean 1
# make predict data

from random import random, seed
from copy import deepcopy

alpha_arr = [i for i in range(3, 13)]

train_predict = []
test_predict = []

x_train = []
x_test = []
y_train = []
y_test = []

for turn_num in turn_vari:
    train_predict.append([])
    test_predict.append([])

    x_train_ele, x_test_ele, y_train_ele, y_test_ele = train_test_split(\
        x_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
        y_data.query("turn_num==%d" % turn_num).drop("turn_num", axis=1),
        random_state=0
    )
    x_train.append(x_train_ele)
    x_test.append(x_test_ele)
    y_train.append(y_train_ele)
    y_test.append(y_test_ele)

    for alpha in alpha_arr:
        model = Ridge(\
            alpha=alpha,
            solver="lbfgs",
            positive=True,
            random_state=0
        )
        model.fit(x_train_ele, y_train_ele)

        train_predict[-1].append(model.predict(x_train_ele))
        test_predict[-1].append(model.predict(x_test_ele))

In [74]:
# weighted mean 2
# local search

GENERATION = 100
CHILDLEN = 30
seed(0)

weight = [random() for i in range(len(alpha_arr))]

min_MAE = 0xffff

for generation in range(GENERATION):
    seed(generation)
    print("\r%4d/%4d MAE: %d" % (generation + 1, GENERATION, min_MAE), end="")
    for child in range(CHILDLEN):
        train_predict_keep = []
        test_predict_keep = []
        train_MAE_sum = 0
        test_MAE_sum = 0
        weight_candi = deepcopy(weight)
        for i in range(len(weight_candi)):
            weight_candi[i] += (0.5 - random()) / 2

        for i in range(len(turn_vari)):
            for j in range(len(alpha_arr)):
                train_predict_weight = np.array([0] * len(train_predict[i][j]))
                test_predict_weight = np.array([0] * len(test_predict[i][j]))
                for k in range(len(train_predict_weight)):
                    train_predict_weight[k] += train_predict[i][j][k] * weight_candi[j]
                for k in range(len(test_predict_weight)):
                    test_predict_weight[k] += test_predict[i][j][k] * weight_candi[j]
            
            train_predict_weight = train_predict_weight / sum(weight_candi)
            test_predict_weight = test_predict_weight / sum(weight_candi)
            train_predict_keep.append(train_predict_weight)
            test_predict_keep.append(test_predict_weight)

            train_MAE_sum += mean_absolute_error(train_predict_weight, y_train[i])
            test_MAE_sum += mean_absolute_error(test_predict_weight, y_test[i])

        if test_MAE_sum < min_MAE and generation != GENERATION - 1:
            min_MAE = test_MAE_sum
            train_predict_ans = deepcopy(train_predict_keep)
            test_predict_ans = deepcopy(test_predict_keep)
            weight_next = deepcopy(weight_candi)
    
    weight = deepcopy(weight_next)

 100/ 100 MAE: 157

In [75]:
# weighted mean 3
# make diagram

train_MAE = []
test_MAE = []

for i in range(len(turn_vari)):
    train_MAE.append(mean_absolute_error(train_predict_ans[i], y_train[i]))
    test_MAE.append(mean_absolute_error(test_predict_ans[i], y_test[i]))

two_plot(
    turn_vari,
    [train_MAE, test_MAE],
    "turn",
    "MAE",
    "MAE of weighted mean (max generation is %d)" % GENERATION,
    "fig"
)

In [76]:
# weighted mean 4
# output result

print(weight)

[-0.1064985666709149, 0.3163889075346738, -0.10411753741468255, -0.8111324426281548, 0.22063493037623882, 0.1415032823705678, -0.3051317188273051, 0.06266523938940216, 0.5196314675991498, 0.8612372013999511]
